In [1]:
import requests
import json
from IPython.display import JSON
from keys import *

In [2]:
#be careful not to run too often as there is a maximum of 200 api calls per month

url = "https://aerodatabox.p.rapidapi.com/flights/airports/icao/LFPG/2025-09-01T08:00/2025-09-01T20:00"

querystring = {"withLeg":"true","direction":"Arrival","withCancelled":"true","withCodeshared":"true","withCargo":"true","withPrivate":"true","withLocation":"false"}

headers = {
    'x-rapidapi-host': "aerodatabox.p.rapidapi.com",
    'x-rapidapi-key': flights_key
    }

response = requests.request("GET", url, headers=headers, params=querystring)

In [3]:
# Comprueba si el código de estado HTTP de la respuesta es 200 (200 significa "OK").
if response.status_code == 200:
    new_data = response.json()

    # Inicia un bloque para manejar errores potenciales al leer el archivo.
    try:
        with open("data.json", "r") as json_file:
            existing_data = json.load(json_file)

    # Captura errores si el archivo no existe o su contenido no es JSON válido.
    except (FileNotFoundError, json.decoder.JSONDecodeError):
        existing_data = []
    
    # Agrega el nuevo dato obtenido de la API a la lista de datos existentes.
    existing_data.append(new_data)
    
    # Abre el archivo "data.json" en modo escritura ("w"), lo que sobrescribe su contenido.
    with open("data.json", "w") as json_file:
        json.dump(existing_data, json_file, indent=4)
        print("Data appended to data.json file.")

# Si el código de estado no fue 200, imprime un mensaje de error con el código de estado HTTP fallido
else:
    print("Failed to retrieve data from the API. Status code:", response.status_code)

Data appended to data.json file.


In [4]:
# Convierte la respuesta de la API a un diccionario JSON
flight_arrivals = response.json()

# Itera sobre los vuelos en la respuesta
for flight in flight_arrivals:
    flight_arrivals['arrivals'][0]['arrival']['scheduledTime']
    flight_arrivals['arrivals'][0]['number']
    flight_arrivals['arrivals'][0]['arrival']['terminal']
    flight_arrivals['arrivals'][0]['departure']['airport']['name']
    flight_arrivals['arrivals'][0]['aircraft']['model']
    flight_arrivals['arrivals'][0]['airline']['name']

In [5]:
import pandas as pd
from numpy import nan

flight_arrivals = response.json()

# Define una función que toma los datos de UN SOLO vuelo ('flights') y los formatea.
def flight_extraction(flights):
    flights_df = pd.DataFrame({
        'scheduled_arrival_time': [flights['arrival']['scheduledTime']['utc']],  
        'flight_number': [flights['number']],
        'from_airport': [flights['departure']['airport']['name']],  
        'airline': [flights['airline']['name']],
        'aircraft': [flights['aircraft']['model']],
        'icao_code': ['LFPG'] 
    })
    return flights_df

# Convierte cada vuelo en un dataFrame con "flight_extraction" y luego los concatena en un único dataFrame
flight_arrivals_df = pd.concat(
    [flight_extraction(flight) for flight in flight_arrivals['arrivals']],
    ignore_index=True  # 🆕 NUEVO: Evita índices duplicados
)

# Convertir scheduled_arrival_time a formato datetime para MySQL
flight_arrivals_df['scheduled_arrival_time'] = pd.to_datetime(flight_arrivals_df['scheduled_arrival_time'])

# Muestra las primeras 5 filas del dataframe.
print(flight_arrivals_df.head())

     scheduled_arrival_time flight_number from_airport          airline  \
0 2025-09-01 06:00:00+00:00       AF 7371         Lyon       Air France   
1 2025-09-01 07:05:00+00:00        DL 262     New York  Delta Air Lines   
2 2025-09-01 06:25:00+00:00       U2 2433       London          easyJet   
3 2025-09-01 06:05:00+00:00       AF 1733        Milan       Air France   
4 2025-09-01 06:35:00+00:00       AF 1381       London       Air France   

          aircraft icao_code  
0      Airbus A320      LFPG  
1   Boeing 767-400      LFPG  
2      Airbus A319      LFPG  
3  Airbus A220-300      LFPG  
4  Airbus A220-300      LFPG  


In [6]:
import sqlalchemy
import pymysql

# connection details for the local mysql database
schema = "gans"
host = "127.0.0.1"
user = "root"
password = "ElMckenzie11$$$"  
port = 3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [8]:
# send the flight data to the database
flight_arrivals_df.to_sql('flight_arrival', if_exists='append', con=con, index=False)


466